In [1]:
! git clone https://github.com/hdspgroup/DeepLDSIR
%cd DeepLDSIR

Cloning into 'DeepLDSIR'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 100 (delta 37), reused 57 (delta 14), pack-reused 0
Receiving objects: 100% (100/100), 32.89 MiB | 11.28 MiB/s, done.
Resolving deltas: 100% (37/37), done.
/content/DeepLDSIR


In [2]:
import sys
sys.path.insert(0, './models')

In [3]:
import tensorflow as tf
import numpy as np
import scipy.io as sio

from utils import *
from models.Alpha8L import *
from matplotlib import pyplot as plt

(1, 512, 512, 31)

In [7]:
data_path = "./data/Image_28.mat"

RGB = [22, 12, 4]
h = sio.loadmat(data_path)['img']
INPUT_SIZE = h.shape         
print(INPUT_SIZE)           

(512, 512, 31)


In [8]:
from dataset import get_pipeline, DataGen

data_path = "./data"
val_path = "./data"

dataset_params = dict(
    input_size = INPUT_SIZE,
    batch_size = 1,
    buffer_size = 1,
    cache_dir = '/content/data',
)

val_params = dict(
    input_size = INPUT_SIZE,
    batch_size = 1,
    buffer_size = 1,
    cache_dir = '',
)


train_ds = get_pipeline(data_path, **dataset_params)
val_ds = get_pipeline(val_path, **val_params)

In [9]:
import os
import tensorflow as tf
import numpy as np
import scipy.io as sio

from utils import *
from models.Alpha8L import *
from metrics import prior_loss, psnr
# from admm import ADMM as algADMM

input_size = (512, 512, 31)
coded_aperture = np.load('ca.npy')

main_model = HSI_net(coded_aperture, input_size=input_size,  pretrained_weights=None)
autoencoder = main_model.priorA

model_params = {
    'loss' : ['mse' , prior_loss],
    'loss_weights' : [1, 0.1],
    'optimizer' : tf.keras.optimizers.Adam(learning_rate=1e-3, amsgrad=False),
    'metrics' : {'decode' : psnr},
}


autoencoder.compile(**model_params)
autoencoder.summary()

Model: "PriorAutoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 512, 512, 31)]    0         
                                                                 
 conv1 (Conv2D)              (None, 512, 512, 64)      17920     
                                                                 
 conv2 (Conv2D)              (None, 512, 512, 64)      36928     
                                                                 
 conv3 (Conv2D)              (None, 512, 512, 32)      18464     
                                                                 
 conv4 (Conv2D)              (None, 512, 512, 32)      9248      
                                                                 
 conv5 (Conv2D)              (None, 512, 512, 16)      4624      
                                                                 
 encode (Conv2D)             (None, 512, 512, 8)  

In [11]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

weights_path = os.path.join('.', 'weights','Alpha8L_toy.h5')

callbacks = [
    ReduceLROnPlateau(monitor="val_decode_psnr", factor=0.9, patience=15,
                      verbose=1, mode="max",) ,
    ModelCheckpoint(weights_path, monitor='val_decode_psnr', verbose=1,
                    save_best_only=True, save_weights_only=True, mode='max')
]

# autoencoder.load_weights(weights_path)
autoencoder.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=callbacks)

Epoch 1/100
      1/Unknown - 12s 12s/step - loss: 0.0301 - decode_loss: 0.0297 - TV_loss: 0.0038 - decode_psnr: 15.2737
Epoch 1: val_decode_psnr improved from -inf to 17.02072, saving model to ./weights/Alpha8L_toy.h5
1/1 [==============================] - 13s 13s/step - loss: 0.0301 - decode_loss: 0.0297 - TV_loss: 0.0038 - decode_psnr: 15.2737 - val_loss: 0.0202 - val_decode_loss: 0.0199 - val_TV_loss: 0.0036 - val_decode_psnr: 17.0207 - lr: 0.0010
Epoch 2/100
1/1 [==============================] - ETA: 0s - loss: 0.0202 - decode_loss: 0.0199 - TV_loss: 0.0036 - decode_psnr: 17.0207
Epoch 2: val_decode_psnr did not improve from 17.02072
1/1 [==============================] - 0s 286ms/step - loss: 0.0202 - decode_loss: 0.0199 - TV_loss: 0.0036 - decode_psnr: 17.0207 - val_loss: 0.0626 - val_decode_loss: 0.0623 - val_TV_loss: 0.0032 - val_decode_psnr: 12.0546 - lr: 0.0010
Epoch 3/100
1/1 [==============================] - ETA: 0s - loss: 0.0626 - decode_loss: 0.0623 - TV_loss: 0.0032 